In [114]:
import math

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

train_data = pd.read_csv('dataset/price_data_tr.csv')
heldout_data = pd.read_csv('dataset/price_data_val.csv')
test_data = pd.read_csv('dataset/price_data_ts.csv')

In [115]:
#print(train_data.shape) #(12968, 21)
#print(heldout_data.shape) #(4323, 21)
#print(test_data.shape) #(4322, 21)

# data.columns
# 'id', 'date', 
# 'price', 
# 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
# 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'

In [116]:
# 최신 연도로 바꾸고 기존 yr_bult, yr_renovated 삭제
train_data['yr_max'] = np.maximum(np.array(train_data.yr_built), np.array(train_data.yr_renovated))
train_data = train_data.drop(['yr_built','yr_renovated'], axis=1)

heldout_data['yr_max'] = np.maximum(np.array(heldout_data.yr_built), np.array(heldout_data.yr_renovated))
heldout_data = heldout_data.drop(['yr_built','yr_renovated'], axis=1)

test_data['yr_max'] = np.maximum(np.array(test_data.yr_built), np.array(test_data.yr_renovated))
test_data = test_data.drop(['yr_built','yr_renovated'], axis=1)

In [117]:
# drop zipcode
train_data['zipcode'] = -train_data['zipcode']
heldout_data['zipcode'] = -heldout_data['zipcode']
test_data['zipcode'] = -test_data['zipcode']

In [118]:
e, pi, r2, re = math.e, math.pi, 2 ** 0.5, math.e ** 0.5

# bathrooms
train_data['bathrooms'] = train_data['bathrooms'].apply(lambda x: x**r2)
# sqft_lot
train_data['sqft_lot'] = train_data['sqft_lot'].apply(lambda x: x**(1/pi))
# floors
train_data['floors'] = train_data['floors'].apply(lambda x: x**(1/(pi**pi)))
# waterfront
train_data['waterfront'] = train_data['waterfront'].apply(lambda x: x**(e**e))
# condition
train_data['condition'] = train_data['condition'].apply(lambda x: x**(pi**2))
# grade
train_data['grade'] = train_data['grade'].apply(lambda x: x**(re**re))
# sqft_basement
train_data['sqft_basement'] = train_data['sqft_basement'].apply(lambda x: x**(r2**r2))
# lat
train_data['lat'] = train_data['lat'].apply(lambda x: x**(1/(pi**pi)))
# sqft_lot15
train_data['sqft_lot15'] = train_data['sqft_lot15'].apply(lambda x: x**(1/pi))
# yr_max
train_data['yr_max'] = train_data['yr_max'].apply(lambda x: x**(e**2))


heldout_data['bathrooms'] = heldout_data['bathrooms'].apply(lambda x: x**r2)
heldout_data['sqft_lot'] = heldout_data['sqft_lot'].apply(lambda x: x**(1/pi))
heldout_data['floors'] = heldout_data['floors'].apply(lambda x: x**(1/(pi**pi)))
heldout_data['waterfront'] = heldout_data['waterfront'].apply(lambda x: x**(e**e))
heldout_data['condition'] = heldout_data['condition'].apply(lambda x: x**(pi**2))
heldout_data['grade'] = heldout_data['grade'].apply(lambda x: x**(re**re))
heldout_data['sqft_basement'] = heldout_data['sqft_basement'].apply(lambda x: x**(r2**r2))
heldout_data['lat'] = heldout_data['lat'].apply(lambda x: x**(1/(pi**pi)))
heldout_data['sqft_lot15'] = heldout_data['sqft_lot15'].apply(lambda x: x**(1/pi))
heldout_data['yr_max'] = heldout_data['yr_max'].apply(lambda x: x**(e**2))


test_data['bathrooms'] = test_data['bathrooms'].apply(lambda x: x**r2)
test_data['sqft_lot'] = test_data['sqft_lot'].apply(lambda x: x**(1/pi))
test_data['floors'] = test_data['floors'].apply(lambda x: x**(1/(pi**pi)))
test_data['waterfront'] = test_data['waterfront'].apply(lambda x: x**(e**e))
test_data['condition'] = test_data['condition'].apply(lambda x: x**(pi**2))
test_data['grade'] = test_data['grade'].apply(lambda x: x**(re**re))
test_data['sqft_basement'] = test_data['sqft_basement'].apply(lambda x: x**(r2**r2))
test_data['lat'] = test_data['lat'].apply(lambda x: x**(1/(pi**pi)))
test_data['sqft_lot15'] = test_data['sqft_lot15'].apply(lambda x: x**(1/pi))
test_data['yr_max'] = test_data['yr_max'].apply(lambda x: x**(e**2))

In [119]:
nor_train = train_data.iloc[:, 3:].apply(lambda x: (x-x.mean()) / x.std())
nor_log_train_price = np.log(train_data.iloc[:, 2:3]).apply(lambda x: (x-x.mean()) / x.std())

nor_heldout = heldout_data.iloc[:, 3:].apply(lambda x: (x-x.mean()) / x.std())
nor_log_heldout_price = np.log(heldout_data.iloc[:, 2:3]).apply(lambda x: (x-x.mean()) / x.std())

nor_test = test_data.iloc[:, 3:].apply(lambda x: (x-x.mean()) / x.std())

In [120]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=True)
train_poly = poly_features.fit_transform(nor_train)
heldout_poly = poly_features.fit_transform(nor_heldout)
test_poly = poly_features.fit_transform(nor_test)

In [121]:
print(nor_train.shape, train_poly.shape)
print(type(train_poly))
print(nor_heldout.shape, heldout_poly.shape)
print(nor_test.shape, test_poly.shape)

(12968, 17) (12968, 171)
<class 'numpy.ndarray'>
(4323, 17) (4323, 171)
(4322, 17) (4322, 171)


In [122]:
train_log_label_mean, train_log_label_std = np.log(train_data['price']).mean(), np.log(train_data['price']).std()
print(train_log_label_mean, train_log_label_std)

heldout_log_label_mean, heldout_log_label_std = np.log(heldout_data['price']).mean(), np.log(heldout_data['price']).std()
print(heldout_log_label_mean, heldout_log_label_std)

uni_log_label_mean = np.log(pd.concat((train_data['price'], heldout_data['price']))).mean()
uni_log_label_std = np.log(pd.concat((train_data['price'], heldout_data['price']))).std()
print(uni_log_label_mean, uni_log_label_std)

13.04810655020148 0.5220080959315785
13.048257499579405 0.5355929705203868
13.048144289728448 0.5254218546587482


In [123]:
train_feats = torch.tensor(train_poly, dtype=torch.float32)
train_labels = torch.tensor(nor_log_train_price.values.reshape(-1, 1), dtype=torch.float32)

heldout_feats = torch.tensor(heldout_poly, dtype=torch.float32)
heldout_labels = torch.tensor(nor_log_heldout_price.values.reshape(-1, 1), dtype=torch.float32)

test_feat = torch.tensor(test_poly, dtype=torch.float32)


In [124]:
print(train_feats.shape, train_poly.shape)
print(heldout_feats.shape, heldout_poly.shape)
print(test_feat.shape, test_poly.shape)

torch.Size([12968, 171]) (12968, 171)
torch.Size([4323, 171]) (4323, 171)
torch.Size([4322, 171]) (4322, 171)


In [125]:
n_train, in_features = train_data.shape[0], train_feats.shape[1]


class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        self.fc1 = nn.Linear(in_features, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 1)
        
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(16)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.dropout(self.relu(self.bn1(self.fc1(x))))
        x = self.dropout(self.relu(self.bn2(self.fc2(x))))
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x
    
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [128]:
learning_rate = 0.001
weight_decay = 0.1
batch_size = 1024
epochs = 500

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = Regressor().to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [130]:
train_features, train_labels = train_feats.to(device), train_labels.to(device)
test_features, test_labels = heldout_feats.to(device), heldout_labels.to(device)

dataset = TensorDataset(train_features, train_labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

train_ls, test_ls = [], []

for epoch in range(epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    train_rmse = criterion(net(train_features), train_labels).item()
    test_rmse = criterion(net(test_features), test_labels).item()
    
    train_ls.append(train_rmse)
    test_ls.append(test_rmse)
    
    if test_rmse < 0.11:
        break
    
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch + 1} -- train_rmse: {train_rmse:.4f}, test_rmse: {test_rmse:.4f}')

epoch: 10 -- train_rmse: 0.0438, test_rmse: 0.1382
epoch: 20 -- train_rmse: 0.0447, test_rmse: 0.1462
epoch: 30 -- train_rmse: 0.0452, test_rmse: 0.1603
epoch: 40 -- train_rmse: 0.0443, test_rmse: 0.1626
epoch: 50 -- train_rmse: 0.0444, test_rmse: 0.1605
epoch: 60 -- train_rmse: 0.0419, test_rmse: 0.1522
epoch: 70 -- train_rmse: 0.0428, test_rmse: 0.1556
epoch: 80 -- train_rmse: 0.0431, test_rmse: 0.1576
epoch: 90 -- train_rmse: 0.0423, test_rmse: 0.1512
epoch: 100 -- train_rmse: 0.0433, test_rmse: 0.1660
epoch: 110 -- train_rmse: 0.0443, test_rmse: 0.1513
epoch: 120 -- train_rmse: 0.0423, test_rmse: 0.1617
epoch: 130 -- train_rmse: 0.0422, test_rmse: 0.1590
epoch: 140 -- train_rmse: 0.0443, test_rmse: 0.1569
epoch: 150 -- train_rmse: 0.0425, test_rmse: 0.1740
epoch: 160 -- train_rmse: 0.0416, test_rmse: 0.1453
epoch: 170 -- train_rmse: 0.0401, test_rmse: 0.1819
epoch: 180 -- train_rmse: 0.0408, test_rmse: 0.1490
epoch: 190 -- train_rmse: 0.0402, test_rmse: 0.1580
epoch: 200 -- train_r

In [ ]:
predicts = net(train_features).to('cpu').detach().numpy()
uni_predicts = np.exp(predicts * train_log_label_std + train_log_label_mean)
train_labels_ = np.exp(train_labels.cpu() * train_log_label_std + train_log_label_mean)
print(rmse(uni_predicts, np.array(train_labels_)))

plt.plot(train_ls, label="Train")
plt.plot(test_ls, label="Test")
plt.legend()
plt.show()

print(f'{train_ls[-1]:0.4f}, {test_ls[-1]:0.4f}')

In [210]:
trains = []
helds = []
unis = []


for i in range(3000):
    predicts = net(test_features).to('cpu').detach().numpy()

    train_predicts = np.exp(predicts * train_log_label_std + train_log_label_mean)
    held_predicts = np.exp(predicts * heldout_log_label_std + heldout_log_label_mean)
    uni_predicts = np.exp(predicts * uni_log_label_std + uni_log_label_mean)
        
    rmse_train = rmse(train_predicts, heldout_data.iloc[:, 2:3].values)
    rmse_held = rmse(held_predicts, heldout_data.iloc[:, 2:3].values)
    rmse_uni = rmse(uni_predicts, heldout_data.iloc[:, 2:3].values)
    
    trains.append(rmse_train)
    helds.append(rmse_held)
    unis.append(rmse_uni)

print(np.array(trains).mean())
print(np.array(helds).mean())
print(np.array(unis).mean())

finals = []

for i in range(3000):
    test_predicts = net(test_feat.to('cuda')).cpu().detach().numpy()
    final_predict = np.exp(test_predicts * uni_log_label_std + uni_log_label_mean)
    finals.append(final_predict)
np.array(finals).mean(axis = 0)

# 131588.9882797251
# 131751.391756751
# 131443.0004292868

131588.9882797251
131751.391756751
131443.0004292868


array([[716624.3 ],
       [494908.06],
       [209662.36],
       ...,
       [891221.75],
       [541079.44],
       [423587.75]], dtype=float32)

In [211]:
test_data = pd.read_csv('dataset/price_data_ts.csv')
test_data['price'] = np.array(finals).mean(axis = 0)

test_data['id'] = test_data['id'].apply(lambda x : str(x) if len(str(x)) == 10 else '0' + str(x) if len(str(x)) == 9 else '00' + str(x))
test_data['id'] = test_data['id'].astype(str) + test_data['date'].astype(str)
submission = pd.concat([test_data['id'], test_data['price']], axis=1)
submission

,id,price
0,700010085020140926T000000,716624.312500
1,403740028020140923T000000,494908.062500
2,142340016020140618T000000,209662.359375
3,776740006020141119T000000,460679.562500
4,946590050020140617T000000,584058.375000
...,...,...
4317,733822037020141006T000000,320622.500000
4318,681910015020140721T000000,642553.187500
4319,082405914020140527T000000,891221.750000
4320,051450009020140513T000000,541079.437500


In [212]:
submission.to_csv('submission.csv', index=False)

In [191]:
def extract_middle(net, x):
    x = net.dropout(net.relu(net.fc1(x)))
    x = net.dropout(net.relu(net.fc2(x)))
    x = net.dropout(net.relu(net.fc3(x)))
    x = net.dropout(net.relu(net.fc4(x)))
    x = net.dropout(net.relu(net.fc5(x)))
    x = net.fc6(x)
    return x

In [192]:
train_feats.shape

torch.Size([12968, 17])

In [193]:
extract_middle(net, train_feats.to('cuda')).shape

torch.Size([12968, 64])

In [194]:
train_labels.flatten().shape

torch.Size([12968])

In [195]:
net = net.cuda() ; train_feats = train_feats.cuda()

In [196]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
model = RandomForestRegressor()
middle_value = extract_middle(net, train_feats)
middle_value = middle_value.cpu()
middle_value = middle_value.detach().numpy()

train_labels_cpu = train_labels.cpu()
train_labels_cpu = train_labels_cpu.detach().numpy()

model.fit(middle_value, train_labels_cpu.flatten())

RandomForestRegressor()

In [197]:
# val_labels_cpu = train_labels.cpu()
# val_labels_cpu = train_labels_cpu.detach().numpy()
middle_val = extract_middle(net, test_features)
middle_val = middle_val.cpu()
middle_val = middle_val.detach().numpy()

predicts = model.predict(middle_val)
train_predicts = np.exp(predicts * train_log_label_std + train_log_label_mean)
held_predicts = np.exp(predicts * heldout_log_label_std + heldout_log_label_mean)
uni_predicts = np.exp(predicts * uni_log_label_std + uni_log_label_mean)

tmse = np.sqrt(mean_squared_error(train_predicts, heldout_data.iloc[:, [2]].values))
hmse = np.sqrt(mean_squared_error(held_predicts, heldout_data.iloc[:, [2]].values))
umse = np.sqrt(mean_squared_error(uni_predicts, heldout_data.iloc[:, [2]].values))

print('평균제곱근오차', tmse)
print('평균제곱근오차', hmse)
print('평균제곱근오차', umse)

평균제곱근오차 213644.33121815647
평균제곱근오차 213296.51417173128
평균제곱근오차 213472.30924589932
